In [1]:
%cd ..

/home/ubuntu/dev/pepper


In [2]:
from copy import deepcopy

import torch
import numpy as np

import mmcv
from mmcv import Config

In [3]:
# %load_ext autoreload
# %autoreload 2

In [4]:
from pepper.datasets import build_dataset, build_dataloader, build_sampler

In [5]:
# load dataset configuration file

cfg_file = "configs/image/_base_/datasets/market1501.py"
# cfg_file = "tests/configs/_base_/datasets/mini_market1501.py"

cfg = Config.fromfile(cfg_file)
print(cfg.pretty_text)

img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile', to_float32=True),
    dict(type='ResizeOrRandom2DTranslation', size=(256, 128), prob=0.5),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(
        type='RandomErasing',
        erase_prob=0.5,
        min_area_ratio=0.02,
        max_area_ratio=0.4),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Collect', keys=['img', 'gt_label']),
    dict(type='FormatBundle')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(256, 128), interpolation='bilinear'),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]
dat

In [6]:
# get train datast
dataset = build_dataset(cfg.data.train)

print("dataset size:", len(dataset))
print("# of pids:", dataset.NUM_PIDS)
print("# of camids:", dataset.NUM_CAMIDS)

dataset size: 12936
# of pids: 751
# of camids: 6


In [7]:
batch_size = 32
num_instances = 4

shuffle = False

rank0_args = dict(
    dataset=dataset,
    num_replicas=2,
    rank=0,
)

rank1_args = dict(
    dataset=dataset,
    num_replicas=2,
    rank=1,
)

sampler_cfg = dict(
    # type="InfiniteBalancedIdentityDistributedSampler",
    type="InfiniteNaiveIdentityDistributedSampler",
    batch_size=batch_size,
    num_instances=num_instances,
    shuffle=shuffle,
    round_up=True,
    seed=0,
)

rank0_sampler = build_sampler(sampler_cfg, default_args=rank0_args)

rank1_sampler = build_sampler(sampler_cfg, default_args=rank1_args)

data_info = deepcopy([info["img_info"] for info in dataset.data_infos])

In [10]:
rank0_iter = iter(rank0_sampler)
rank1_iter = iter(rank1_sampler)

print("rank:\tbatch\tpid\tcamid")

for i in range(batch_size * 1):
    
    idx0 = next(rank0_iter)
    idx1 = next(rank1_iter)
    
    info0 = data_info[idx0]
    info1 = data_info[idx1]
    
    print("rank{}\t{}\t{}\t{}".format(0, i % batch_size, info0['pid'], info0['camid']))
    print("rank{}\t{}\t{}\t{}".format(1, i % batch_size, info1['pid'], info1['camid']))

rank:	batch	pid	camid
rank0	0	474	2
rank1	0	545	0
rank0	1	474	4
rank1	1	545	5
rank0	2	474	4
rank1	2	545	2
rank0	3	474	0
rank1	3	545	1
rank0	4	113	5
rank1	4	206	4
rank0	5	113	4
rank1	5	206	0
rank0	6	113	4
rank1	6	206	2
rank0	7	113	1
rank1	7	206	2
rank0	8	266	1
rank1	8	661	5
rank0	9	266	2
rank1	9	661	4
rank0	10	266	5
rank1	10	661	5
rank0	11	266	5
rank1	11	661	0
rank0	12	31	3
rank1	12	219	4
rank0	13	31	3
rank1	13	219	3
rank0	14	31	0
rank1	14	219	0
rank0	15	31	3
rank1	15	219	2
rank0	16	549	5
rank1	16	231	2
rank0	17	549	1
rank1	17	231	2
rank0	18	549	5
rank1	18	231	0
rank0	19	549	2
rank1	19	231	2
rank0	20	533	0
rank1	20	6	5
rank0	21	533	2
rank1	21	6	2
rank0	22	533	1
rank1	22	6	3
rank0	23	533	0
rank1	23	6	0
rank0	24	721	1
rank1	24	348	5
rank0	25	721	5
rank1	25	348	1
rank0	26	721	0
rank1	26	348	5
rank0	27	721	1
rank1	27	348	4
rank0	28	14	5
rank1	28	117	2
rank0	29	14	2
rank1	29	117	2
rank0	30	14	1
rank1	30	117	5
rank0	31	14	5
rank1	31	117	2


In [10]:
indices = []

for s in iter_samples:
    indices.append(s)

KeyboardInterrupt: 

In [11]:
len(indices)

796480

In [90]:
(12936 / 32) / 2

202.125

In [12]:
len(indices) / 32

47.0

In [93]:
12936 / 64

202.125